In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

# Initialize Spark session
spark = SparkSession.builder.appName("CustomerOffersPipeline").getOrCreate()

# Azure Storage Configuration - Directly in Code (Without Secrets)
storage_account_name = "storageaccountofferpulse"
storage_account_key = "RLo88cac943ee86vcqHEOHDDUOobZ4hzN0ikBKXOOg/iV+IqvMTSaXpBrhoz2rVaQO+iSn63B692+AStEY4ppw=="

#storage_account_name = dbutils.secrets.get(scope="my_scope", key="storageaccountname")
#storage_account_key = dbutils.secrets.get(scope="my_scope", key="storageaccountkey")

spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_key)

container_name = "offerpulseraw"
input_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net"

# Set Spark Configuration for Azure Storage
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_key)

# Define Schemas
demographic_schema = StructType([
    StructField("cust_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("mobile_number", StringType(), True),
    StructField("pan_no", StringType(), True),
    StructField("passport_no", StringType(), True)
])

credit_bureau_schema = StructType([
    StructField("cust_id", StringType(), True),
    StructField("credit_score", IntegerType(), True),
    StructField("income", IntegerType(), True)
])

offer_schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("offer_id", StringType(), True),
    StructField("offer_type", StringType(), True),
    StructField("offer_status", StringType(), True)
])

# Read Data from Azure Blob Storage
demographic_df = spark.read.schema(demographic_schema).json(f"{input_path}/Demographic_Data/*.json", multiLine=True)
credit_bureau_df = spark.read.schema(credit_bureau_schema).json(f"{input_path}/CIBIL_Data/*.json", multiLine=True)
offer_df = spark.read.schema(offer_schema).json(f"{input_path}/Offer_Data/*.json", multiLine=True)

# Show Sample DataFrames
demographic_df.show()
credit_bureau_df.show()
offer_df.show()

